In [1]:
import numpy as np
import pandas as pd
from river import evaluate
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sail.transformers.river.preprocessing import StandardScaler
from sail.models.torch.rnn import RNNRegressor

#### Load Data


In [2]:
X = pd.read_csv("../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
learner_gru = RNNRegressor(
    input_units=12,
    output_units=1,
    hidden_units=100,
    n_hidden_layers=3,
    lr=0.001,
    cell_type="GRU",
    verbose=0
)

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", learner_gru),
]
sail_pipeline = SAILPipeline(steps=steps, scoring="R2", verbosity_level=1, verbosity_interval=100)

#### Train Test Split


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Start Incremental Training


In [6]:
y_preds = []
y_true = []
batch_size = 10

for start in range(0, 5000, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_pipeline.partial_fit(X, y)


    
>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.034s, Batch Size=10, P_Score=0]            

    
>> Epoch: 100 | Samples Seen: 990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=-.195]            

    
>> Epoch: 200 | Samples Seen: 1990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=0.239]            

    
>> Epoch: 300 | Samples Seen: 2990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Sco

#### Save SAIL Pipeline


In [7]:
sail_pipeline.save(".")

[2023-10-02 00:50:33:111] - INFO - SAIL (PyTorch) - Model saved successfully.


'./sail_pipeline'

#### Load SAIL Pipeline


In [8]:
sail_new_pipeline = SAILPipeline.load(".")

SAIL Pipeline Progressive Score: 100%========== [Points: 4990/4990, Elapsed:0.0098s, Metric=R2, Batch Size=4990, P_Score=0.526]


In [9]:
y_preds = []
y_true = []
batch_size = 10

for start in range(5001, 10768, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_new_pipeline.partial_fit(X, y)

  epoch    train_loss     dur
-------  ------------  ------
    501       72.9539  0.0011
    502      566.4787  0.0011
    503      132.8276  0.0011
    504      115.9440  0.0013
    505      245.5162  0.0012
    506      170.1505  0.0013
    507      136.3387  0.0011
    508      441.2717  0.0012
    509      118.4934  0.0013
    510      180.9164  0.0010
    511      327.4562  0.0012
    512      367.7494  0.0010
    513      180.3380  0.0011
    514      165.7492  0.0010
    515      192.4044  0.0011
    516      302.7406  0.0013
    517      273.7800  0.0010
    518      251.5001  0.0010
    519      248.4970  0.0010
    520      276.3235  0.0012
    521      599.9944  0.0010
    522      254.7390  0.0012
    523      208.6125  0.0011
    524      398.7694  0.0011
    525      144.0548  0.0013
    526      478.4481  0.0013
    527      229.7571  0.0014
    528      100.2355  0.0011
    529      203.9711  0.0011
    530       39.9040  0.0010
    531      139.7032  0.0010
    532   


    
>> Epoch: 600 | Samples Seen: 5990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.004s, Batch Size=10]      

    600      430.7782  0.0042


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.010s, Batch Size=10, P_Score=0.56]            

    601      336.8963  0.0023
    602      142.4220  0.0013
    603      260.9496  0.0013
    604      211.0201  0.0012
    605      482.9416  0.0010
    606      151.5855  0.0012
    607      447.2303  0.0012
    608      396.9992  0.0011
    609      278.3117  0.0013
    610       88.2425  0.0011
    611      451.3175  0.0011
    612      522.4875  0.0013
    613       54.5248  0.0012
    614      265.1602  0.0012
    615      338.1915  0.0012
    616      363.1442  0.0009
    617      800.0979  0.0012
    618      153.4140  0.0011
    619      273.0910  0.0010
    620      630.5610  0.0012
    621      376.4488  0.0014
    622      248.1106  0.0014
    623      274.0689  0.0010
    624      111.2421  0.0011
    625      232.5437  0.0011
    626      294.4535  0.0011
    627      604.3905  0.0011
    628      248.6763  0.0012
    629       82.2511  0.0011
    630       80.4971  0.0010
    631      266.3600  0.0011
    632      281.4179  0.0010
    633      458.5069  0.0010
    634   

...


    
>> Epoch: 1000 | Samples Seen: 9990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

   1000      381.6263  0.0027


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.629]            

   1001      312.1396  0.0053
   1002      217.7592  0.0036
   1003      361.2621  0.0014
   1004      162.3637  0.0012
   1005       40.6156  0.0011
   1006      119.1513  0.0013
   1007      302.9351  0.0012
   1008      113.5313  0.0011
   1009      146.1936  0.0012
   1010      632.6141  0.0010
   1011      176.1833  0.0011
   1012      173.5949  0.0011
   1013      113.5121  0.0012
   1014       96.1610  0.0012
   1015      614.6171  0.0012
   1016      106.5000  0.0011
   1017      307.6331  0.0010
   1018      255.4276  0.0010
   1019      177.5762  0.0010
   1020      298.8627  0.0010
   1021      233.9860  0.0010
   1022      292.8723  0.0012
   1023      245.9397  0.0012
   1024      196.9597  0.0013
   1025      258.2346  0.0011
   1026       71.7609  0.0011
   1027      196.3288  0.0012
   1028      587.0775  0.0010
   1029      319.2612  0.0011
   1030      185.8290  0.0010
   1031       80.0009  0.0010
   1032      699.5176  0.0010
   1033      333.9524  0.0010
   1034   

   1035      385.1044  0.0012
   1036      221.8911  0.0012
   1037      311.7576  0.0011
   1038      599.7544  0.0011
   1039      225.6103  0.0011
   1040      460.1644  0.0010
   1041      215.6809  0.0010
   1042      120.1493  0.0012
   1043      164.9901  0.0011
   1044      508.0969  0.0009
   1045      263.8515  0.0012
   1046       40.1944  0.0011
   1047      413.7855  0.0012
   1048       21.7896  0.0011
   1049      134.2711  0.0010
   1050      126.0769  0.0011
   1051      183.9503  0.0011
   1052      139.6402  0.0010
   1053      307.5391  0.0011
   1054      362.5526  0.0011
   1055      110.3634  0.0010
   1056      259.9938  0.0011
   1057      268.5780  0.0010
   1058      322.8316  0.0012
   1059      152.8616  0.0010
   1060      509.4525  0.0010
   1061      143.2292  0.0011
   1062      228.3846  0.0012
   1063       79.0228  0.0011
   1064      152.2517  0.0010
   1065      253.1306  0.0009
   1066      272.1347  0.0010
   1067      385.6739  0.0009
   1068   

#### Make Prediction on Hold out set


In [10]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):

    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Final Score


In [11]:
sail_new_pipeline.get_progressive_score

0.635659857151349

In [13]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(100)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()